# Intro
The goal of this notebook is to produce an automated valuation tool for Singapore. We make use of the data provided by the Urban Redevelopment Authority (URA) found in the dataset folder.

We will build three different models and each time test their accuracy:
- A hedonic regression model
- An XGboost model
- A random forest regression model

Finally, we will combine these three models into an ensamble method and compare its accuracy to the accuracy of the individual models.

TODO: Build an AVM tool that uses the model (see bottom). Potentially can be turned in a webapp.
TODO: Clean up the code. This notebook is a colation of several seperate scripts and therefore repeats itself unnecessarily.

# Hedonic regression

## Building the model
We use the Ordinary Least Squares (OLS) api from Statsmodels to build a 5 quarter rolling hedonic regression model.

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

dataset = pd.read_csv('datasets/singapore_ura.csv', index_col = 0)
y = dataset['log_price_psf']
    
results_series_list = []
period_list = ['%sQ%s' % (year, qtr) for year in range(2017,2021) for qtr in range(1,5)]
period_list = period_list[:-2]

In [3]:
for start_index in range(0,len(period_list)+1-5):
    window_period_list = period_list[start_index:start_index+5]

    period_filter = dataset['Period_%s' % window_period_list[0]] == 1
    for i in [1,2,3,4]:
        period_filter = period_filter | (dataset['Period_%s' % window_period_list[i]] == 1)

    y_target = y[period_filter]
    X_target = dataset[period_filter]
    X_columns = [c for c in dataset.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % period for period in window_period_list[1:5]]
    X_target = X_target[X_columns]
    X_target = sm.add_constant(X_target)

    model = sm.OLS(y_target,X_target)
    results = model.fit()

    result_series = results.params
    pvalue_series = results.pvalues
    pvalue_series.index = ['pvalue_%s' % idx for idx in pvalue_series.index]
    result_series = result_series.append(pvalue_series)
    result_series = result_series.append(pd.Series([results.rsquared], index = ['rsquared']))
    result_series = result_series.append(pd.Series([results.rsquared_adj], index = ['rsquared_adj']))
    result_series = result_series.append(pd.Series([results.nobs], index = ['nobs']))

    results_series_list.append(result_series)
    results.save("temp_files/sgp_rolling_%s.pkl" % window_period_list[-1])
    print("...saved to sgp_rolling_%s.pkl" % window_period_list[-1])

results_df = pd.concat(results_series_list,axis = 1)
results_df.columns = period_list[0:len(period_list)+1-5]
results_df.to_csv('temp_files/sgp_rolling.csv')
print("...saved to temp_files/sgp_rolling.csv")

...saved to sgp_rolling_2018Q1.pkl
...saved to sgp_rolling_2018Q2.pkl
...saved to sgp_rolling_2018Q3.pkl
...saved to sgp_rolling_2018Q4.pkl
...saved to sgp_rolling_2019Q1.pkl
...saved to sgp_rolling_2019Q2.pkl
...saved to sgp_rolling_2019Q3.pkl
...saved to sgp_rolling_2019Q4.pkl
...saved to sgp_rolling_2020Q1.pkl
...saved to sgp_rolling_2020Q2.pkl
>>>saved to sgp_rolling.csv


In [6]:
index_dict = {}
for start_index in range(0,len(period_list)+1-5):
    window_period_list = period_list[start_index:start_index+5]
    index_list = [100.0]
    for i in [1,2,3,4]:
        index_list.append(100.0 * np.exp(results_df[window_period_list[0]]['Period_%s' % window_period_list[i]]))    
    index_dict[window_period_list[0]] = index_list

final_index_list = []
for period in sorted(index_dict.keys()):
    if not len(final_index_list):
        final_index_list = index_dict[period]
    else:
        new_index_value = index_dict[period][-1] * final_index_list[-1] / index_dict[period][-2]
        final_index_list.append(new_index_value)

final_index_df = pd.DataFrame(final_index_list, index = period_list, columns = ['Index'])
final_index_df.to_csv('temp_files/sgp_rolling_index.csv')
print("...saved to temp_files/sgp_rolling_index.csv")

...saved to temp_files/sgp_rolling_index.csv


## 1.2. Testing accuracy
We use the Mean Absolute Percentage Error (MAPE) and Median Absolute Percentage Error (MdAPE) to determine the accuracy of our model.

In [9]:
from statsmodels.regression.linear_model import OLSResults
from copy import deepcopy

df = pd.read_csv('datasets/singapore_ura.csv', index_col=0)

per_ls = ['%sQ%s' % (yr, qtr) for yr in range(2017,2021) for qtr in range(1,5)]
per_ls = per_ls[:-2]

ape_ls = []
for start in range(0, len(per_ls)-5):
    window = per_ls[start:start+5]
    last = window[-1]
    out = per_ls[start+5]

    test = deepcopy(df[df['Period_%s' % out] == 1])
    test['Period_%s' % last] = 1
    test['Period_%s' % out] = 0

    cols = [c for c in test.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % x for x in window[1:]]

    X = test[cols]
    X = sm.add_constant(X, has_constant = 'add')

    results = OLSResults.load('temp_files/sgp_rolling_%s.pkl' % last)
    pred_psf = np.exp(results.predict(X))
    act_psf = np.exp(test['log_price_psf'])

    ape = np.abs(pred_psf - act_psf) / act_psf
    ape = ape.to_frame()
    ape['Period'] = out
    ape_ls.append(ape)

ape_df = pd.concat(ape_ls)

MdAPE = ape_df[0].median()
MAPE = ape_df[0].mean()

ape95 = ape_df[0][ape_df[0] < ape_df[0].quantile(0.95)]
MdAPE95 = ape95.median()
MAPE95 = ape95.mean()

pct_in_20 = (ape_df[0] < 0.2).sum() / ape_df[0].count()
pct_in_10 = (ape_df[0] < 0.1).sum() / ape_df[0].count()
pct_in_5 = (ape_df[0] < 0.05).sum() / ape_df[0].count()

print('MdAPE: %.1f%%' % (MdAPE*100))
print('MAPE: %.1f%%' % (MAPE*100))
print('MdAPE within %.1f%%, 95%% of the time' % (MdAPE95*100))
print('MAPE within %.1f%%, 95%% of the time' % (MAPE95*100))
print('%0.1f%% of predictions within 20%% of observed values' % (100.0*pct_in_20))
print('%0.1f%% of predictions within 10%% of observed values' % (100.0*pct_in_10))
print('%0.1f%% of predictions within 5%% of observed values' % (100.0*pct_in_5))

MdAPE: 9.3%
MAPE: 12.1%
MdAPE within 8.7%, 95% of the time
MAPE within 10.4%, 95% of the time
82.5% of predictions within 20% of observed values
52.9% of predictions within 10% of observed values
29.4% of predictions within 5% of observed values


# 2. XGBoost

## 2.1. Building the model
We use the XGBRegressor api from xgboost to build a 5 quarter rolling xgboost model.

In [11]:
import xgboost as xgb
import pickle

dataset = pd.read_csv('datasets/singapore_ura.csv', index_col = 0)
y = dataset['log_price_psf']

results_series_list = []
period_list = ['%sQ%s' % (year, qtr) for year in range(2017,2021) for qtr in range(1,5)]
period_list = period_list[:-2]

for start_index in range(0,len(period_list)+1-5):
    window_period_list = period_list[start_index:start_index+5]

    period_filter = dataset['Period_%s' % window_period_list[0]] == 1
    for i in [1,2,3,4]:
        period_filter = period_filter | (dataset['Period_%s' % window_period_list[i]] == 1)

    y_target = y[period_filter]
    X_target = dataset[period_filter]
    X_columns = [c for c in dataset.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % period for period in window_period_list[1:5]]
    X_target = X_target[X_columns]

    xg_reg = xgb.XGBRegressor()
    xg_reg.fit(X_target, y_target)
    pickle.dump(xg_reg, open('temp_files/sgp_rolling_xgb_%s.pkl' % window_period_list[-1], 'wb'))
    print('...saved to temp_files/sgp_rolling_xgb_%s.pkl' % window_period_list[-1])

print('Done')

...saved to temp_files/sgp_rolling_xgb_2018Q1.pkl
...saved to temp_files/sgp_rolling_xgb_2018Q2.pkl
...saved to temp_files/sgp_rolling_xgb_2018Q3.pkl
...saved to temp_files/sgp_rolling_xgb_2018Q4.pkl
...saved to temp_files/sgp_rolling_xgb_2019Q1.pkl
...saved to temp_files/sgp_rolling_xgb_2019Q2.pkl
...saved to temp_files/sgp_rolling_xgb_2019Q3.pkl
...saved to temp_files/sgp_rolling_xgb_2019Q4.pkl
...saved to temp_files/sgp_rolling_xgb_2020Q1.pkl
...saved to temp_files/sgp_rolling_xgb_2020Q2.pkl
Done


## 2.2. Testing accuracy
We use the Mean Absolute Percentage Error (MAPE) and Median Absolute Percentage Error (MdAPE) to determine the accuracy of our model.

In [12]:
df = pd.read_csv('datasets/singapore_ura.csv', index_col=0)

per_ls = ['%sQ%s' % (yr, qtr) for yr in range(2017,2021) for qtr in range(1,5)]
per_ls = per_ls[:-2]

ape_ls = []
for start in range(0, len(per_ls)-5):
    window = per_ls[start:start+5]
    last = window[-1]
    out = per_ls[start+5]

    test = deepcopy(df[df['Period_%s' % out] == 1])
    test['Period_%s' % last] = 1
    test['Period_%s' % out] = 0

    cols = [c for c in test.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % x for x in window[1:]]
    X = test[cols]

    results = pickle.load(open('temp_files/sgp_rolling_xgb_%s.pkl' % last, 'rb'))
    pred_psf = np.exp(results.predict(X))
    act_psf = np.exp(test['log_price_psf'])

    ape = np.abs(pred_psf - act_psf) / act_psf
    ape = ape.to_frame()
    ape['Period'] = out
    ape_ls.append(ape)

ape_df = pd.concat(ape_ls)
ape_df.columns = [0, 'Period']

MdAPE = ape_df[0].median()
MAPE = ape_df[0].mean()

ape95 = ape_df[0][ape_df[0] < ape_df[0].quantile(0.95)]
MdAPE95 = ape95.median()
MAPE95 = ape95.mean()

pct_in_20 = (ape_df[0] < 0.2).sum() / ape_df[0].count()
pct_in_10 = (ape_df[0] < 0.1).sum() / ape_df[0].count()
pct_in_5 = (ape_df[0] < 0.05).sum() / ape_df[0].count()

print('MdAPE: %.1f%%' % (MdAPE*100))
print('MAPE: %.1f%%' % (MAPE*100))
print('MdAPE within %.1f%%, 95%% of the time' % (MdAPE95*100))
print('MAPE within %.1f%%, 95%% of the time' % (MAPE95*100))
print('%0.1f%% of predictions within 20%% of observed values' % (100.0*pct_in_20))
print('%0.1f%% of predictions within 10%% of observed values' % (100.0*pct_in_10))
print('%0.1f%% of predictions within 5%% of observed values' % (100.0*pct_in_5))

MdAPE: 6.4%
MAPE: 9.1%
MdAPE within 5.9%, 95% of the time
MAPE within 7.7%, 95% of the time
89.3% of predictions within 20% of observed values
66.5% of predictions within 10% of observed values
41.9% of predictions within 5% of observed values


# 3. Random forrests regression

## 3.1. Building the model
We use the Random Forrest Regressor api from SKlearn to build a 5 quarter rolling random forrest regression model.

In [14]:
import sklearn.ensemble as sk

dataset = pd.read_csv('datasets/singapore_ura.csv', index_col = 0)
y = dataset['log_price_psf']

results_series_list = []
period_list = ['%sQ%s' % (year, qtr) for year in range(2017,2021) for qtr in range(1,5)]
period_list = period_list[:-2]

for start_index in range(0,len(period_list)+1-5):
    window_period_list = period_list[start_index:start_index+5]

    period_filter = dataset['Period_%s' % window_period_list[0]] == 1
    for i in [1,2,3,4]:
        period_filter = period_filter | (dataset['Period_%s' % window_period_list[i]] == 1)

    y_target = y[period_filter]
    X_target = dataset[period_filter]
    X_columns = [c for c in dataset.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % period for period in window_period_list[1:5]]
    X_target = X_target[X_columns]

    rf_reg = sk.RandomForestRegressor()
    rf_reg.fit(X_target, y_target)
    pickle.dump(rf_reg, open('temp_files/sgp_rolling_rf_%s.pkl' % window_period_list[-1], 'wb'))
    print('...savend to temp_files/sgp_rolling_rf_%s.pkl' % window_period_list[-1])

print('Done')

...savend to temp_files/sgp_rolling_rf_2018Q1.pkl
...savend to temp_files/sgp_rolling_rf_2018Q2.pkl
...savend to temp_files/sgp_rolling_rf_2018Q3.pkl
...savend to temp_files/sgp_rolling_rf_2018Q4.pkl
...savend to temp_files/sgp_rolling_rf_2019Q1.pkl
...savend to temp_files/sgp_rolling_rf_2019Q2.pkl
...savend to temp_files/sgp_rolling_rf_2019Q3.pkl
...savend to temp_files/sgp_rolling_rf_2019Q4.pkl
...savend to temp_files/sgp_rolling_rf_2020Q1.pkl
...savend to temp_files/sgp_rolling_rf_2020Q2.pkl
Done


## 3.2. Testing accuracy
We use the Mean Absolute Percentage Error (MAPE) and Median Absolute Percentage Error (MdAPE) to determine the accuracy of our model.

In [15]:
df = pd.read_csv('datasets/singapore_ura.csv', index_col=0)

per_ls = ['%sQ%s' % (yr, qtr) for yr in range(2017,2021) for qtr in range(1,5)]
per_ls = per_ls[:-2]

ape_ls = []
for start in range(0, len(per_ls)-5):
    window = per_ls[start:start+5]
    last = window[-1]
    out = per_ls[start+5]

    test = deepcopy(df[df['Period_%s' % out] == 1])
    test['Period_%s' % last] = 1
    test['Period_%s' % out] = 0

    cols = [c for c in test.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % x for x in window[1:]]
    X = test[cols]

    results = pickle.load(open('temp_files/sgp_rolling_rf_%s.pkl' % last, 'rb'))
    pred_psf = np.exp(results.predict(X))
    act_psf = np.exp(test['log_price_psf'])

    ape = np.abs(pred_psf - act_psf) / act_psf
    ape = ape.to_frame()
    ape['Period'] = out
    ape_ls.append(ape)

ape_df = pd.concat(ape_ls)
ape_df.columns = [0, 'Period']

MdAPE = ape_df[0].median()
MAPE = ape_df[0].mean()

ape95 = ape_df[0][ape_df[0] < ape_df[0].quantile(0.95)]
MdAPE95 = ape95.median()
MAPE95 = ape95.mean()

pct_in_20 = (ape_df[0] < 0.2).sum() / ape_df[0].count()
pct_in_10 = (ape_df[0] < 0.1).sum() / ape_df[0].count()
pct_in_5 = (ape_df[0] < 0.05).sum() / ape_df[0].count()

print('MdAPE: %.1f%%' % (MdAPE*100))
print('MAPE: %.1f%%' % (MAPE*100))
print('MdAPE within %.1f%%, 95%% of the time' % (MdAPE95*100))
print('MAPE within %.1f%%, 95%% of the time' % (MAPE95*100))
print('%0.1f%% of predictions within 20%% of observed values' % (100.0*pct_in_20))
print('%0.1f%% of predictions within 10%% of observed values' % (100.0*pct_in_10))
print('%0.1f%% of predictions within 5%% of observed values' % (100.0*pct_in_5))

MdAPE: 5.6%
MAPE: 9.0%
MdAPE within 5.2%, 95% of the time
MAPE within 7.4%, 95% of the time
88.5% of predictions within 20% of observed values
68.0% of predictions within 10% of observed values
46.5% of predictions within 5% of observed values


# 4. Ensemble method

## 4.1. Building the model
It is clear that the XGBoost and Random Forest model outperform the Hedonic Regression model. Can we improve upon their individual scores by combining them? We will test this by making use of the SKlearn package.

In [18]:
df = pd.read_csv('datasets/singapore_ura.csv', index_col=0)

per_ls = ['%sQ%s' % (yr, qtr) for yr in range(2017,2021) for qtr in range(1,5)]
per_ls = per_ls[:-2]

weights = [0, 0.23, 0.24, 0.25, 0.26, 0.27, 1]

ape_dict = {}
for w in weights:
    ape_dict[w] = []

ape_ls = []
for start in range(0, len(per_ls)-5):
    window = per_ls[start:start+5]
    last = window[-1]
    out = per_ls[start+5]

    test = deepcopy(df[df['Period_%s' % out] == 1])
    test['Period_%s' % last] = 1
    test['Period_%s' % out] = 0

    cols = [c for c in test.columns if not c.startswith('Period_') and not c in ['log_price_psf']] + ['Period_%s' % x for x in window[1:]]
    X = test[cols]

    xgb_results = pickle.load(open('temp_files/sgp_rolling_xgb_%s.pkl' % last, 'rb'))
    xgb_pred_psf = np.exp(xgb_results.predict(X))

    rf_results = pickle.load(open('temp_files/sgp_rolling_rf_%s.pkl' % last, 'rb'))
    rf_pred_psf = np.exp(rf_results.predict(X))
    act_psf = np.exp(test['log_price_psf'])

    for w in weights:
        pred_psf = w * xgb_pred_psf + (1.0-w) * rf_pred_psf

        ape = np.abs(pred_psf - act_psf) / act_psf
        ape = ape.to_frame()
        ape['Period'] = out
        ape.rename(columns={'log_price_psm':'ape'}, inplace=True)
        
        ape_dict[w].append(ape)

## 4.2. Testing accuracy
We use the Mean Absolute Percentage Error (MAPE) and Median Absolute Percentage Error (MdAPE) to determine the accuracy of our model.

In [19]:
for w in weights:
    ape_df = pd.concat(ape_dict[w])
    ape_df.columns = [0, 'Period']
    
    MdAPE = ape_df[0].median()
    MAPE = ape_df[0].mean()
    
    ape95 = ape_df[0][ape_df[0] < ape_df[0].quantile(0.95)]
    MdAPE95 = ape95.median()
    MAPE95 = ape95.mean()
    
    pct_in_20 = (ape_df[0] < 0.2).sum() / ape_df[0].count()
    pct_in_10 = (ape_df[0] < 0.1).sum() / ape_df[0].count()
    pct_in_5 = (ape_df[0] < 0.05).sum() / ape_df[0].count()
    
    print('w: ', w)
    print('MdAPE: %.1f%%' % (MdAPE*100))
    print('MAPE: %.1f%%' % (MAPE*100))
    print('MdAPE within %.1f%%, 95%% of the time' % (MdAPE95*100))
    print('MAPE within %.1f%%, 95%% of the time' % (MAPE95*100))
    print('%0.1f%% of predictions within 20%% of observed values' % (100.0*pct_in_20))
    print('%0.1f%% of predictions within 10%% of observed values' % (100.0*pct_in_10))
    print('%0.1f%% of predictions within 5%% of observed values' % (100.0*pct_in_5))
    print('-------------------------')

w:  0
MdAPE: 5.6%
MAPE: 9.0%
MdAPE within 5.2%, 95% of the time
MAPE within 7.4%, 95% of the time
88.5% of predictions within 20% of observed values
68.0% of predictions within 10% of observed values
46.5% of predictions within 5% of observed values
-------------------------
w:  0.23
MdAPE: 5.6%
MAPE: 8.7%
MdAPE within 5.2%, 95% of the time
MAPE within 7.2%, 95% of the time
89.5% of predictions within 20% of observed values
68.9% of predictions within 10% of observed values
46.5% of predictions within 5% of observed values
-------------------------
w:  0.24
MdAPE: 5.6%
MAPE: 8.7%
MdAPE within 5.2%, 95% of the time
MAPE within 7.2%, 95% of the time
89.5% of predictions within 20% of observed values
68.9% of predictions within 10% of observed values
46.4% of predictions within 5% of observed values
-------------------------
w:  0.25
MdAPE: 5.6%
MAPE: 8.7%
MdAPE within 5.2%, 95% of the time
MAPE within 7.2%, 95% of the time
89.6% of predictions within 20% of observed values
68.9% of predi

# Conclusion

Unsurprisingly XGBoost and Random Forrest outperformed Hedonic Regression. However, less expected is that Random Forrest slightly outperformed XGBoost and that an ensemble method of the two only improved marginally on the Random Forrest model.

Our best performing model is therefore the ensemble model made up of a 24% contribution from XGBoost model and a 76% contribution from Random Forrest model.

# AVM

Fill in the parameters of your property below and run the cell. A predicted value for your property will be returned, based on the ensamble model described above.